# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [2]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")
# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,buckie,57.6757,-2.9624,31.71,76,8,5.10,GB,1699571046
1,1,coahuayana de hidalgo,18.7000,-103.6583,85.03,71,85,4.41,MX,1699571046
2,2,west island,-12.1568,96.8225,78.78,89,75,14.97,CC,1699571047
3,3,sinabang,2.4803,96.3801,78.12,85,99,3.60,ID,1699571047
4,4,tazacorte,28.6290,-17.9293,70.77,70,100,18.99,ES,1699571047


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [7]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True, 
    tiles = "OSM", 
    frame_width = 700, 
    frame_height = 500,
    size = "Humidity",
    scale = 0.5,
    color = "City"
)

# Display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
city_ideal = city_data_df.loc[(city_data_df["Max Temp"] < 85) & (city_data_df["Max Temp"] >= 70) & (city_data_df["Wind Speed"] <= 10) & \
                              (city_data_df["Cloudiness"] == 0)]

# Drop any rows with null values
city_ideal_df = city_ideal.dropna()

# Display sample data
city_ideal_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
104,104,kafue,-15.7691,28.1814,73.87,55,0,6.87,ZM,1699571089
105,105,arraial do cabo,-22.9661,-42.0278,76.96,69,0,9.22,BR,1699571089
118,118,vila velha,-20.3297,-40.2925,76.95,74,0,10.00,BR,1699571091
124,124,zouerate,22.7187,-12.4521,73.35,25,0,7.00,MR,1699571091
136,136,khachrod,23.4167,75.2833,70.29,45,0,5.10,IN,1699571094
162,162,saint-pierre,-21.3393,55.4781,73.08,73,0,6.91,RE,1699571110
180,180,as samawah,31.3320,45.2944,73.51,31,0,4.76,IQ,1699571117
187,187,shush,32.1942,48.2436,72.45,34,0,2.82,IR,1699571119
198,198,ghanzi,-21.5667,21.7833,79.99,21,0,8.03,BW,1699571135
246,246,boiling springs,35.0465,-81.9818,75.85,56,0,4.61,US,1699571153


### Step 3: Create a new DataFrame called `hotel_df`.

In [19]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_ideal_df.loc[:,["City","Country","Lat","Lng"]].copy()


hotel_df
# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Hotel Name
104,kafue,ZM,-15.7691,28.1814,
105,arraial do cabo,BR,-22.9661,-42.0278,
118,vila velha,BR,-20.3297,-40.2925,
124,zouerate,MR,22.7187,-12.4521,
136,khachrod,IN,23.4167,75.2833,
162,saint-pierre,RE,-21.3393,55.4781,
180,as samawah,IQ,31.3320,45.2944,
187,shush,IR,32.1942,48.2436,
198,ghanzi,BW,-21.5667,21.7833,
246,boiling springs,US,35.0465,-81.9818,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [36]:
# Set parameters to search for a hotel
radius = 10000
categories = "accommodation.hotel"
limit = 1

params = {
    "categories":categories,
    "limit":limit,
    "apiKey":geoapify_key    
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = hotel_df.loc[index, "Lat"]
    lng = hotel_df.loc[index, "Lng"]
    
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make an API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
kafue - nearest hotel: Lwanginga guesthouse
arraial do cabo - nearest hotel: No hotel found
vila velha - nearest hotel: Hotel Prainha
zouerate - nearest hotel: فندق تيرس زمور
khachrod - nearest hotel: No hotel found
saint-pierre - nearest hotel: Tropic Hotel
as samawah - nearest hotel: فندق قصر الغدير
shush - nearest hotel: هتل آپادانا
ghanzi - nearest hotel: Ghanzi farmhouse
boiling springs - nearest hotel: Days Inn by Wyndham Spartanburg
illizi - nearest hotel: بوناقة
salalah - nearest hotel: Muscat International Hotel
taoudenni - nearest hotel: No hotel found
aioun - nearest hotel: فندق العيون
bandar abbas - nearest hotel: هتل دریا
jacksonville - nearest hotel: Hyatt Regency Jacksonville Riverfront
baixo guandu - nearest hotel: No hotel found
sirte - nearest hotel: فندق المدينة-سرت
parkes - nearest hotel: Commercial Hotel
aswan - nearest hotel: Yaseen hotel
lospalos - nearest hotel: Hotel Roberto Carlos
remire-montjoly - nearest hotel: Complexe Belova


,City,Country,Lat,Lng,Hotel Name
104,kafue,ZM,-15.7691,28.1814,Lwanginga guesthouse
105,arraial do cabo,BR,-22.9661,-42.0278,No hotel found
118,vila velha,BR,-20.3297,-40.2925,Hotel Prainha
124,zouerate,MR,22.7187,-12.4521,فندق تيرس زمور
136,khachrod,IN,23.4167,75.2833,No hotel found
162,saint-pierre,RE,-21.3393,55.4781,Tropic Hotel
180,as samawah,IQ,31.3320,45.2944,فندق قصر الغدير
187,shush,IR,32.1942,48.2436,هتل آپادانا
198,ghanzi,BW,-21.5667,21.7833,Ghanzi farmhouse
246,boiling springs,US,35.0465,-81.9818,Days Inn by Wyndham Spartanburg


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [35]:
%%capture --no-display

vacation_map = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 900,
    frame_height = 500,
    color = "City",
    hover_cols = ['Hotel Name', 'Country']
)

# Display the map
vacation_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Hotel Name,Country)